### IMPORTS

In [1]:
import pandas as pd
from dhis2 import Api
import requests
from io import StringIO
import sqlite3

C:\Users\rkapo\AppData\Local\Temp\ipykernel_21968\3306669705.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


#### DEFINE UTILITY FUNCTIONS and VARIABLES

In [2]:
# con = sqlite3.connect("db/irs_final.db")

user = ""
password = ""
api = Api('', user, password)
ou = 'Ot1P2HjvtvL'
ouMode = 'DESCENDANTS'

db_name = 'irs_final.db'
# load DF to db table
def load_data(tbl_name, df):
    try:
        with sqlite3.connect(f"db/{db_name}") as con:
            df.to_sql(tbl_name, con, if_exists="replace")
            print(f"Data loaded to {tbl_name} in DB")
            
        con.close()
    except sqlite3.Error as e:
        print("Something went wrong...!!!")
        print()
        print(e)

# Read data from Db Table into Df
def read_data(tbl_name):
    try:
        with sqlite3.connect(f"db/{db_name}") as con:
            print(f"Reading data from {tbl_name} in DB")
            df = pd.read_sql_query(f"SELECT * from {tbl_name}", con)
        con.close()
    except sqlite3.Error as e:
        print("Something went wrong...!!!")
        print()
        print(e)    
    
    return df

#### GET IRS PROGRAM DATA ELEMENTS

In [68]:
program = 'N02UmHMXkNQ'
# Get program stages from Program
params={'fields':'name,id,programStages[id]'}

stage_res = api.get(f'programs/{program}', params=params)
data = stage_res.json()
program_stage_id = data['programStages'][0]['id']
print("Program Stage ID:", program_stage_id)

# get Data elements on program stage:
params= {'fields':'name,id,programStageDataElements[dataElement[id,name]]'}
elements_res = stage = api.get(f'programStages/{program_stage_id}', params=params)
elements = elements_res.json()
# print(elements['programStageDataElements'])

# Extracting 'dataElement' from each dictionary in the list
elements_data = elements['programStageDataElements']
elements_data = [entry['dataElement'] for entry in elements_data]

# Creating a DataFrame
elements_df = pd.DataFrame(elements_data, columns=['name', 'id'])

# write df to database table
load_data("irs_de", elements_df)

Program Stage ID: TX5i6nl6WPD
Data loaded to irs_de in DB


In [69]:
irs_de = read_data("irs_de")
irs_de.head(2)

Reading data from irs_de in DB


,index,name,id
0,0,IRS-Bottles or Sachets,VfRv14EfqT9
1,1,IRS-Children <5 years Protected,Lxa6iNrKaov


#### GET IRS PROGRAM DATA ELEMENTS

In [70]:
program = 'Eu3XRyBGz43'
# Get program stages from Program
params={'fields':'name,id,programStages[id]'}

stage_res = api.get(f'programs/{program}', params=params)
data = stage_res.json()
program_stage_id = data['programStages'][0]['id']
print("Program Stage ID:", program_stage_id)

# get Data elements on program stage:
params= {'fields':'name,id,programStageDataElements[dataElement[id,name]]'}
elements_res = stage = api.get(f'programStages/{program_stage_id}', params=params)
elements = elements_res.json()
# print(elements['programStageDataElements'])

# Extracting 'dataElement' from each dictionary in the list
elements_data = elements['programStageDataElements']
elements_data = [entry['dataElement'] for entry in elements_data]

# Creating a DataFrame
elements_df = pd.DataFrame(elements_data, columns=['name', 'id'])

# write df to database table
load_data("microplan_de", elements_df)

Program Stage ID: C7481b4XpGJ
Data loaded to microplan_de in DB


In [71]:
microplan_de = read_data("microplan_de")
microplan_de.head(2)

Reading data from microplan_de in DB


,index,name,id
0,0,IRS-Total Eligible Structures,IuObyB7ZxeZ
1,1,IRS-Total Population,xlrVTPcVcCw


#### GET ORG UNITS FROM SERVER

In [72]:

def org_unit_processing(data):
    try:
        # Create a pandas DataFrame
        print("Cleaning org units....")
        df = pd.DataFrame(data)

        # drop all rows that are empty in the facility and facility_uid columns
        df = df.dropna(subset=['facility', 'facility_uid'])

        df = df[~df.apply(lambda row: row.astype(str).str.startswith(('zz Test', 'zm Test')).any(), axis=1)]

        # Add a new column 'country' and populate it with 'Zambia'
        df.insert(0, 'country', 'Zambia')

        # Sort the DataFrame by 'province', 'district', and 'facility' in ascending order
        df = df.sort_values(by=['province', 'district', 'facility'], ascending=[True, True, True])   

        # Remove province Prefix from Province and district as well as Province and District suffix
        def clean_province(value):
            # Remove the first 2 characters
            cleaned_value = value[2:]
            # Remove the word "province"
            cleaned_value = cleaned_value.replace('Province', '')
            # Strip leading and trailing whitespace
            return cleaned_value.strip()

        def clean_district(value):
            # Remove the first 2 characters
            cleaned_value = value[2:]
            # Remove the word "province"
            cleaned_value = cleaned_value.replace('District', '')
            # Strip leading and trailing whitespace
            return cleaned_value.strip()

        # Apply the function to the 'province' column
        df['province'] = df['province'].apply(clean_province)
        df['district'] = df['district'].apply(clean_district)

        # Assuming df is your DataFrame
        df.drop_duplicates(inplace=True)

        # Drop rows where facility_uid is NaN, None, or an empty string
        org_units = df.dropna(subset=['facility_uid'])
        org_units = org_units[org_units['facility_uid'].str.strip() != '']

        # Drop the index column
        org_units.reset_index(drop=True, inplace=True)

        # Save the DataFrame to a SQL lite db
        load_data("org_units", org_units)

        return "success"

    except Exception as e:
        print(f"Error occurred while processing org units: {str(e)}")
        return "fail"

def pull_orgs(api):
    try:
        print("Pulling Org units from DHIS2 ....")
        response = api.get_sqlview('hsfcjU12wEM', var={'valueType': 'INTEGER'}, merge=True)
        return org_unit_processing(response)

    except RequestException as e:
        print("Something went wrong..!!")
        print()
        print(e)
        return "fail"


result = pull_orgs(api)  # Pass the API object here
if result == "success":
    print("Org unit processing successful")
else:
    print("Org unit processing failed")

Pulling Org units from DHIS2 ....
Cleaning org units....
Data loaded to org_units in DB
Org unit processing successful


In [73]:
org_units = read_data("org_units")
org_units.head(4)

Reading data from org_units in DB


,index,country,province,district,facility,facility_uid
0,0,Zambia,Central,Chibombo,ce Chamakubi Health Post,pXhz0PLiYZX
1,1,Zambia,Central,Chibombo,ce Chibombo Rural Health Centre,di3U5u7F8Y3
2,2,Zambia,Central,Chibombo,ce Chikobo Rural Health Centre,B5LpoYehUfI
3,3,Zambia,Central,Chibombo,ce Chikumbi ZNS Rural Health Centre,zAuN8bxMOMB


#### GET IRS PROGRAM DATA

In [74]:
# Load data Elements from data file
elements_df = read_data("irs_de")

program = 'N02UmHMXkNQ'
#params={'orgUnit':ou, 'ouMode':ouMode, 'program':program,'startDate':'2023-9-12','endDate':'2023-9-30'}
params={'orgUnit':ou, 'ouMode':ouMode, 'program':program, 'startDate':'2023-09-01','fields':'storedBy,event,eventDate,created,lastUpdated,programStage,orgUnit,deleted,dataValues[dataElement,value]'}

print(f"Pulling IRS data from DHIS2 for IRS PROFRAM with ID {program}")
all_pages = api.get_paged('events', params=params, page_size=1000, merge=True)
events = all_pages['events']
print(f"{len(events)} Records retried from DHIS2")

# Creating a DataFrame
events_df = pd.json_normalize(events, 'dataValues', ['event','eventDate','created','lastUpdated','programStage','orgUnit','deleted'])

#change date types
events_df['eventDate'] = pd.to_datetime(events_df['eventDate'])
events_df['created'] = pd.to_datetime(events_df['created'])
events_df['lastUpdated'] = pd.to_datetime(events_df['lastUpdated'])

# Pivot the DataFrame
pivot_df = events_df.pivot_table(index=['event','eventDate','created','lastUpdated','programStage','orgUnit','deleted'],
                          columns='dataElement', values='value', aggfunc='first')

# Reset the index
pivot_df.reset_index(inplace=True)

# Create a dictionary mapping ids to names
id_to_name = dict(zip(elements_df['id'], elements_df['name']))

pivot_df.reset_index(drop=True)
# Rename columns of events_df using the dictionary
pivot_df.rename(columns=id_to_name, inplace=True)


#Delete deleted rows
pivot_df = pivot_df[pivot_df['deleted'] != True]


pivot_df.reset_index(inplace=True)
# sum_df = sum_df.loc[:, ~sum_df.columns.duplicated()]  # Drop duplicate columns

# Check if 'level_0' exists and remove it
if 'level_0' in pivot_df.columns:
    pivot_df.drop(columns=['level_0'], inplace=True)

# Save the DataFrame to a SQL lite db
load_data("irs_raw_data", pivot_df)


Reading data from irs_de in DB
Pulling IRS data from DHIS2 for IRS PROFRAM with ID N02UmHMXkNQ
3294 Records retried from DHIS2
Data loaded to irs_raw_data in DB


In [75]:
irs_raw_data = read_data("irs_raw_data")
irs_raw_data.head(4)

Reading data from irs_raw_data in DB


,level_0,index,event,eventDate,created,lastUpdated,programStage,orgUnit,deleted,IRS-Reason Rooms Not Sprayed: Refused,...,IRS-Total Population Protected,IRS-Pregnant Women Protected,IRS-Insecticide Units Received,IRS-Total Structures Sprayed,IRS-Team Number,IRS-Males Protected,IRS-Reason Rooms Not Sprayed: Locked,IRS-Insecticide Units Missing,IRS-Eligible Rooms Found,IRS-Insecticide
0,0,0,A0W3DLEQqZ1,2023-12-09 00:00:00,2023-12-27 21:52:40.688000,2023-12-27 21:52:40.688000,TX5i6nl6WPD,rsd6wyJI5EZ,0,6,...,466,15,40,95,1,222,10,0,362,Fludora Fusion
1,1,1,A1UmS7PTZVZ,2023-12-28 00:00:00,2023-12-28 10:33:44.813000,2023-12-28 10:33:44.813000,TX5i6nl6WPD,wOuXKB8pWy5,0,0,...,499,5,15,155,15,255,0,0,253,Fludora Fusion
2,2,2,A1kNSN5typi,2023-11-25 00:00:00,2023-11-29 09:29:51.703000,2023-11-29 09:29:51.703000,TX5i6nl6WPD,kIW1nq2PSFD,0,20,...,464,17,25,83,Team 3,205,19,0,346,Fludora Fusion
3,3,3,A3Cb46oVSab,2024-01-01 00:00:00,2024-02-25 15:42:50.327000,2024-02-25 15:42:50.327000,TX5i6nl6WPD,lxlV95Lr6kQ,0,0,...,178,3,16,45,4,82,1,0,104,Fludora Fusion


#### GET MICROPLAN PROGRAM DATA

In [76]:
# Load data Elements from data file
elements_df = read_data("microplan_de")

program = 'Eu3XRyBGz43'
#params={'orgUnit':ou, 'ouMode':ouMode, 'program':program,'startDate':'2023-9-12','endDate':'2023-9-30'}
params={'orgUnit':ou, 'ouMode':ouMode, 'program':program, 'startDate':'2023-9-1','fields':'storedBy,event,eventDate,created,lastUpdated,programStage,orgUnit,deleted,dataValues[dataElement,value]'}

print(f"Pulling IRS Microplan data from DHIS2 for MCIROPLAN PROFRAM with ID {program}")
all_pages = api.get_paged('events', params=params, page_size=1000, merge=True)
microplan_events = all_pages['events']
print(f"{len(events)} Records retried from DHIS2")

# Creating a DataFrame
microplan_df = pd.json_normalize(microplan_events, 'dataValues', ['event','eventDate','created','lastUpdated','programStage','orgUnit','deleted'])

#change date types
microplan_df['eventDate'] = pd.to_datetime(microplan_df['eventDate'])
microplan_df['created'] = pd.to_datetime(microplan_df['created'])
microplan_df['lastUpdated'] = pd.to_datetime(microplan_df['lastUpdated'])


# Printing the DataFrame
#microplan_df.head()

# Pivot the DataFrame
pivot_df = microplan_df.pivot_table(index=['event','eventDate','created','lastUpdated','programStage','orgUnit','deleted'],
                          columns='dataElement', values='value', aggfunc='first')

# Reset the index
pivot_df.reset_index(inplace=True)

# Create a dictionary mapping ids to names
id_to_name = dict(zip(elements_df['id'], elements_df['name']))

pivot_df.reset_index(drop=True)
# Rename columns of events_df using the dictionary
pivot_df.rename(columns=id_to_name, inplace=True)

#Delete deleted rows
pivot_df = pivot_df[pivot_df['deleted'] != True]
pivot_df.drop(["programStage","deleted"], axis = 1, inplace=True)

#change date types
pivot_df['IRS- Campaign Start Date'] = pd.to_datetime(pivot_df['IRS- Campaign Start Date'])
pivot_df['IRS- Campaign End Date'] = pd.to_datetime(pivot_df['IRS- Campaign End Date'])
pivot_df['IRS-Total Eligible  Structures'] = pd.to_numeric(pivot_df['IRS-Total Eligible  Structures'], errors='coerce')
pivot_df['IRS-Total Targeted Eligible Structures'] = pd.to_numeric(pivot_df['IRS-Total Targeted Eligible Structures'], errors='coerce')
pivot_df['IRS-Total Targeted Population'] = pd.to_numeric(pivot_df['IRS-Total Targeted Population'], errors='coerce')
pivot_df['IRS-Total Population'] = pd.to_numeric(pivot_df['IRS-Total Population'], errors='coerce')

# Sort Date on event created date
df_sorted = pivot_df.sort_values(by='created', ascending=False)

# Drop duplicates based on specified columns, keeping the first occurrence (newest record)
# df_no_duplicates = df_sorted.drop_duplicates(subset=['eventDate', 'orgUnit', 'IRS- Campaign Start Date','IRS- Campaign End Date'])

# Group by 'orgUnit' and get the index of the row with the most recent 'lastUpdated'
idx = df_sorted.groupby('orgUnit')['lastUpdated'].idxmax()

# Use the indexes to create a new DataFrame with the most recent rows
final_df = df_sorted.loc[idx]
final_df.drop(["event","eventDate","created","lastUpdated"], axis = 1, inplace=True)


# Save the DataFrame to a SQL lite db
load_data("microplan", final_df)


Reading data from microplan_de in DB
Pulling IRS Microplan data from DHIS2 for MCIROPLAN PROFRAM with ID Eu3XRyBGz43
3294 Records retried from DHIS2
Data loaded to microplan in DB


In [77]:
microplan_data = read_data("microplan")
microplan_data.head(4)

Reading data from microplan in DB


,index,orgUnit,IRS-Total Eligible Structures,IRS- Campaign End Date,IRS-Total Targeted Eligible Structures,IRS- Campaign Start Date,IRS-Total Targeted Population,IRS-Total Population
0,3,A2WG7HbvZ5m,1893,2023-11-06 00:00:00,1893.0,2023-09-26 00:00:00,7519.0,7519.0
1,333,A7YT3iNUs5G,170,2023-11-29 00:00:00,170.0,2023-11-10 00:00:00,1400.0,1899.0
2,218,A87peYAyqsf,2363,2023-11-02 00:00:00,2363.0,2023-09-26 00:00:00,7596.0,7596.0
3,50,A9BC0fz8cxX,3000,2023-12-09 00:00:00,3000.0,2023-11-13 00:00:00,9080.0,9080.0


### CLEAN AND PROCESS IRS PROGRAM DATA

In [79]:
# Load the data into a DataFrame
stage1_df =read_data("irs_raw_data")

#change other data types
print("Changing data types")
stage1_df['IRS-Reason Rooms Not Sprayed: Refused'] = pd.to_numeric(stage1_df['IRS-Reason Rooms Not Sprayed: Refused'], errors='coerce')
stage1_df['IRS-Insecticide Units Used'] = pd.to_numeric(stage1_df['IRS-Insecticide Units Used'], errors='coerce')
stage1_df['IRS-Pregnant Women Sleeping Under Nets'] = pd.to_numeric(stage1_df['IRS-Pregnant Women Sleeping Under Nets'], errors='coerce')
stage1_df['IRS-Total Nets Available'] = pd.to_numeric(stage1_df['IRS-Total Nets Available'], errors='coerce')
stage1_df['IRS-Children <5 Yrs Sleeping Under Nets'] = pd.to_numeric(stage1_df['IRS-Children <5 Yrs Sleeping Under Nets'], errors='coerce')
stage1_df['IRS-Females Protected'] = pd.to_numeric(stage1_df['IRS-Females Protected'], errors='coerce')
stage1_df['IRS-Total People Sleeping Under Nets'] = pd.to_numeric(stage1_df['IRS-Total People Sleeping Under Nets'], errors='coerce')
stage1_df['IRS-Eligible Rooms Sprayed'] = pd.to_numeric(stage1_df['IRS-Eligible Rooms Sprayed'], errors='coerce')
stage1_df['IRS-No of Spray Operators'] = pd.to_numeric(stage1_df['IRS-No of Spray Operators'], errors='coerce')
stage1_df['IRS-Reason Structures Not Sprayed: Other'] = pd.to_numeric(stage1_df['IRS-Reason Structures Not Sprayed: Other'], errors='coerce')
stage1_df['IRS-Insecticides Units Returned Full'] = pd.to_numeric(stage1_df['IRS-Insecticides Units Returned Full'], errors='coerce')
stage1_df['IRS-Reason Rooms Not Sprayed: Sick'] = pd.to_numeric(stage1_df['IRS-Reason Rooms Not Sprayed: Sick'], errors='coerce')
stage1_df['IRS-Reason Structures Not Sprayed: Locked'] = pd.to_numeric(stage1_df['IRS-Reason Structures Not Sprayed: Locked'], errors='coerce')
stage1_df['IRS-Children <5 years Protected'] = pd.to_numeric(stage1_df['IRS-Children <5 years Protected'], errors='coerce')
stage1_df['IRS-Reason Structures Not Sprayed: Funeral'] = pd.to_numeric(stage1_df['IRS-Reason Structures Not Sprayed: Funeral'], errors='coerce')
stage1_df['IRS-Total Structures Not Sprayed'] = pd.to_numeric(stage1_df['IRS-Total Structures Not Sprayed'], errors='coerce')
stage1_df['IRS-Reason Structures Not Sprayed: Refused'] = pd.to_numeric(stage1_df['IRS-Reason Structures Not Sprayed: Refused'], errors='coerce') 
stage1_df['IRS-Total Structures Found'] = pd.to_numeric(stage1_df['IRS-Total Structures Found'], errors='coerce')
stage1_df['IRS-Reason Structures Not Sprayed: Sick'] = pd.to_numeric(stage1_df['IRS-Reason Structures Not Sprayed: Sick'], errors='coerce')
stage1_df['IRS-Total Rooms Not Sprayed'] = pd.to_numeric(stage1_df['IRS-Total Rooms Not Sprayed'], errors='coerce')
stage1_df['IRS-Reason Rooms Not Sprayed: Other'] = pd.to_numeric(stage1_df['IRS-Reason Rooms Not Sprayed: Other'], errors='coerce')
stage1_df['IRS-Total Nets Being Used'] = pd.to_numeric(stage1_df['IRS-Total Nets Being Used'], errors='coerce')
stage1_df['IRS-Reason Structures Not Sprayed: Not Home/Missed'] = pd.to_numeric(stage1_df['IRS-Reason Structures Not Sprayed: Not Home/Missed'], errors='coerce') 
stage1_df['IRS-Total Population Protected'] = pd.to_numeric(stage1_df['IRS-Total Population Protected'], errors='coerce')
stage1_df['IRS-Pregnant Women Protected'] = pd.to_numeric(stage1_df['IRS-Pregnant Women Protected'], errors='coerce')
stage1_df['IRS-Insecticide Units Received'] = pd.to_numeric(stage1_df['IRS-Insecticide Units Received'], errors='coerce')
stage1_df['IRS-Total Structures Sprayed'] = pd.to_numeric(stage1_df['IRS-Total Structures Sprayed'], errors='coerce') 
stage1_df['IRS-Males Protected'] = pd.to_numeric(stage1_df['IRS-Males Protected'], errors='coerce')
stage1_df['IRS-Reason Rooms Not Sprayed: Locked'] = pd.to_numeric(stage1_df['IRS-Reason Rooms Not Sprayed: Locked'], errors='coerce')
stage1_df['IRS-Insecticide Units Missing'] = pd.to_numeric(stage1_df['IRS-Insecticide Units Missing'], errors='coerce')
stage1_df['IRS-Eligible Rooms Found'] = pd.to_numeric(stage1_df['IRS-Eligible Rooms Found'], errors='coerce')


# Drop duplicates based on specified columns, keeping the first occurrence (newest record)
print("Drop duplicates")
df_no_duplicates = stage1_df.drop_duplicates(subset=['eventDate', 'orgUnit', 'IRS-Team Number','IRS-Insecticide'])

# Check if 'level_0' exists and remove it
if 'level_0' in df_no_duplicates.columns:
    df_no_duplicates.drop(columns=['level_0'], inplace=True)

# Save the DataFrame to a SQL lite db
load_data("stage1_irs_data", df_no_duplicates)


Reading data from irs_raw_data in DB
Changing data types
Drop duplicates


C:\Users\rkapo\AppData\Local\Temp\ipykernel_1932\3360309347.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_duplicates.drop(columns=['level_0'], inplace=True)


Data loaded to stage1_irs_data in DB


In [80]:
stage1_df = read_data("stage1_irs_data")
stage1_df.head(4)

Reading data from stage1_irs_data in DB


,level_0,index,event,eventDate,created,lastUpdated,programStage,orgUnit,deleted,IRS-Reason Rooms Not Sprayed: Refused,...,IRS-Total Population Protected,IRS-Pregnant Women Protected,IRS-Insecticide Units Received,IRS-Total Structures Sprayed,IRS-Team Number,IRS-Males Protected,IRS-Reason Rooms Not Sprayed: Locked,IRS-Insecticide Units Missing,IRS-Eligible Rooms Found,IRS-Insecticide
0,0,0,A0W3DLEQqZ1,2023-12-09 00:00:00,2023-12-27 21:52:40.688000,2023-12-27 21:52:40.688000,TX5i6nl6WPD,rsd6wyJI5EZ,0,6.0,...,466.0,15.0,40.0,95.0,1,222.0,10.0,0.0,362.0,Fludora Fusion
1,1,1,A1UmS7PTZVZ,2023-12-28 00:00:00,2023-12-28 10:33:44.813000,2023-12-28 10:33:44.813000,TX5i6nl6WPD,wOuXKB8pWy5,0,0.0,...,499.0,5.0,15.0,155.0,15,255.0,0.0,0.0,253.0,Fludora Fusion
2,2,2,A1kNSN5typi,2023-11-25 00:00:00,2023-11-29 09:29:51.703000,2023-11-29 09:29:51.703000,TX5i6nl6WPD,kIW1nq2PSFD,0,20.0,...,464.0,17.0,25.0,83.0,Team 3,205.0,19.0,0.0,346.0,Fludora Fusion
3,3,3,A3Cb46oVSab,2024-01-01 00:00:00,2024-02-25 15:42:50.327000,2024-02-25 15:42:50.327000,TX5i6nl6WPD,lxlV95Lr6kQ,0,0.0,...,178.0,3.0,16.0,45.0,4,82.0,1.0,0.0,104.0,Fludora Fusion


In [81]:

columns=['event',
         'eventDate',
         'created', 
         'lastUpdated', 
         'programStage', 
         'deleted',
         'IRS-Bottles or Sachets',
         'IRS-Main Spray or Mop Up',
         'IRS-Team Number',
         'IRS-Insecticide'
]
stage1_df.drop(columns=columns, inplace=True)

# Fill al NaN with 0
print("Filling NaN with 0")
stage1_df.fillna(0, inplace=True)


# Covert to Interger
print("Converting interger Data Types")
stage1_df['IRS-Reason Rooms Not Sprayed: Refused'] = stage1_df['IRS-Reason Rooms Not Sprayed: Refused'].astype(int)
stage1_df['IRS-Insecticide Units Used'] = stage1_df['IRS-Insecticide Units Used'].astype(int)
stage1_df['IRS-Pregnant Women Sleeping Under Nets'] = stage1_df['IRS-Pregnant Women Sleeping Under Nets'].astype(int)
stage1_df['IRS-Total Nets Available'] = stage1_df['IRS-Total Nets Available'].astype(int)
stage1_df['IRS-Children <5 Yrs Sleeping Under Nets'] = stage1_df['IRS-Children <5 Yrs Sleeping Under Nets'].astype(int)
stage1_df['IRS-Females Protected'] = stage1_df['IRS-Females Protected'].astype(int)
stage1_df['IRS-Total People Sleeping Under Nets'] = stage1_df['IRS-Total People Sleeping Under Nets'].astype(int)
stage1_df['IRS-Eligible Rooms Sprayed'] = stage1_df['IRS-Eligible Rooms Sprayed'].astype(int)
stage1_df['IRS-No of Spray Operators'] = stage1_df['IRS-No of Spray Operators'].astype(int)
stage1_df['IRS-Reason Structures Not Sprayed: Other'] = stage1_df['IRS-Reason Structures Not Sprayed: Other'].astype(int)
stage1_df['IRS-Insecticides Units Returned Full'] = stage1_df['IRS-Insecticides Units Returned Full'].astype(int) 
stage1_df['IRS-Reason Rooms Not Sprayed: Sick'] = stage1_df['IRS-Reason Rooms Not Sprayed: Sick'].astype(int)
stage1_df['IRS-Reason Structures Not Sprayed: Locked'] = stage1_df['IRS-Reason Structures Not Sprayed: Locked'].astype(int)
stage1_df['IRS-Children <5 years Protected'] = stage1_df['IRS-Children <5 years Protected'].astype(int)
stage1_df['IRS-Reason Structures Not Sprayed: Funeral'] = stage1_df['IRS-Reason Structures Not Sprayed: Funeral'].astype(int) 
stage1_df['IRS-Total Structures Not Sprayed'] = stage1_df['IRS-Total Structures Not Sprayed'].astype(int)
stage1_df['IRS-Reason Structures Not Sprayed: Refused'] = stage1_df['IRS-Reason Structures Not Sprayed: Refused'].astype(int) 
stage1_df['IRS-Total Structures Found'] = stage1_df['IRS-Total Structures Found'].astype(int) 
stage1_df['IRS-Reason Structures Not Sprayed: Sick'] = stage1_df['IRS-Reason Structures Not Sprayed: Sick'].astype(int) 
stage1_df['IRS-Total Rooms Not Sprayed'] = stage1_df['IRS-Total Rooms Not Sprayed'].astype(int)
stage1_df['IRS-Reason Rooms Not Sprayed: Other'] = stage1_df['IRS-Reason Rooms Not Sprayed: Other'].astype(int)
stage1_df['IRS-Total Nets Being Used'] = stage1_df['IRS-Total Nets Being Used'].astype(int)
stage1_df['IRS-Reason Structures Not Sprayed: Not Home/Missed'] = stage1_df['IRS-Reason Structures Not Sprayed: Not Home/Missed'].astype(int) 
stage1_df['IRS-Total Population Protected'] = stage1_df['IRS-Total Population Protected'].astype(int) 
stage1_df['IRS-Pregnant Women Protected'] = stage1_df['IRS-Pregnant Women Protected'].astype(int) 
stage1_df['IRS-Insecticide Units Received'] = stage1_df['IRS-Insecticide Units Received'].astype(int) 
stage1_df['IRS-Total Structures Sprayed'] = stage1_df['IRS-Total Structures Sprayed'].astype(int) 
stage1_df['IRS-Males Protected'] = stage1_df['IRS-Males Protected'].astype(int) 
stage1_df['IRS-Reason Rooms Not Sprayed: Locked'] = stage1_df['IRS-Reason Rooms Not Sprayed: Locked'].astype(int) 
stage1_df['IRS-Insecticide Units Missing'] = stage1_df['IRS-Insecticide Units Missing'].astype(int) 
stage1_df['IRS-Eligible Rooms Found'] = stage1_df['IRS-Eligible Rooms Found'].astype(int)

print("Aggregating data by Org Unit")
sum_df = stage1_df.groupby('orgUnit').sum()
sum_df.reset_index(inplace=True)

# Check if 'level_0' exists and remove it
if 'level_0' in sum_df.columns:
    sum_df.drop(columns=['level_0'], inplace=True)


# Save the DataFrame to a SQL lite db
load_data("stage2_aggregate_irs_data", sum_df)


Filling NaN with 0
Converting interger Data Types
Aggregating data by Org Unit
Data loaded to stage2_aggregate_irs_data in DB


In [82]:
stage2_df = read_data("stage2_aggregate_irs_data")
stage2_df.head(4)

Reading data from stage2_aggregate_irs_data in DB


,level_0,orgUnit,index,IRS-Reason Rooms Not Sprayed: Refused,IRS-Insecticide Units Used,IRS-Pregnant Women Sleeping Under Nets,IRS-Total Nets Available,IRS-Children <5 Yrs Sleeping Under Nets,IRS-Females Protected,IRS-Total People Sleeping Under Nets,...,IRS-Total Nets Being Used,IRS-Reason Structures Not Sprayed: Not Home/Missed,IRS-Total Population Protected,IRS-Pregnant Women Protected,IRS-Insecticide Units Received,IRS-Total Structures Sprayed,IRS-Males Protected,IRS-Reason Rooms Not Sprayed: Locked,IRS-Insecticide Units Missing,IRS-Eligible Rooms Found
0,0,A2WG7HbvZ5m,3176,0,0,0,0,0,3633,0,...,142,0,7329,111,0,1982,3696,0,0,3746
1,1,A7YT3iNUs5G,3640,20,78,36,175,160,904,260,...,175,0,1760,36,100,323,856,2,0,1188
2,2,A87peYAyqsf,2029,0,0,0,0,0,5419,0,...,576,0,10676,320,0,1862,5257,0,0,7470
3,3,ABcUyezNjiu,11994,26,164,91,1874,452,1889,3308,...,1734,0,3624,106,251,828,1735,8,0,2494


## FINAL ORG UNIT + MICROPLAN + AGGREGATED IRS DATA Processing

In [83]:
stage2_df = read_data("stage2_aggregate_irs_data")
org_unit_df = read_data("org_units")
microplan_df = read_data("microplan")

Reading data from stage2_aggregate_irs_data in DB
Reading data from org_units in DB
Reading data from microplan in DB


In [84]:
# CLEAN IRS DATA
stage2_df.reset_index(drop=True)
stage2_df.head()

,level_0,orgUnit,index,IRS-Reason Rooms Not Sprayed: Refused,IRS-Insecticide Units Used,IRS-Pregnant Women Sleeping Under Nets,IRS-Total Nets Available,IRS-Children <5 Yrs Sleeping Under Nets,IRS-Females Protected,IRS-Total People Sleeping Under Nets,...,IRS-Total Nets Being Used,IRS-Reason Structures Not Sprayed: Not Home/Missed,IRS-Total Population Protected,IRS-Pregnant Women Protected,IRS-Insecticide Units Received,IRS-Total Structures Sprayed,IRS-Males Protected,IRS-Reason Rooms Not Sprayed: Locked,IRS-Insecticide Units Missing,IRS-Eligible Rooms Found
0,0,A2WG7HbvZ5m,3176,0,0,0,0,0,3633,0,...,142,0,7329,111,0,1982,3696,0,0,3746
1,1,A7YT3iNUs5G,3640,20,78,36,175,160,904,260,...,175,0,1760,36,100,323,856,2,0,1188
2,2,A87peYAyqsf,2029,0,0,0,0,0,5419,0,...,576,0,10676,320,0,1862,5257,0,0,7470
3,3,ABcUyezNjiu,11994,26,164,91,1874,452,1889,3308,...,1734,0,3624,106,251,828,1735,8,0,2494
4,4,ACa6vpZuNzf,22304,37,343,26,293,146,1547,531,...,264,0,3341,70,413,613,1794,11,0,2206


In [85]:
irs_data =stage2_df[['orgUnit', 'IRS-Total Population Protected','IRS-Total Structures Found','IRS-Total Structures Not Sprayed','IRS-Total Structures Sprayed']].copy()
irs_data.head()

,orgUnit,IRS-Total Population Protected,IRS-Total Structures Found,IRS-Total Structures Not Sprayed,IRS-Total Structures Sprayed
0,A2WG7HbvZ5m,7329,1983,1,1982
1,A7YT3iNUs5G,1760,329,6,323
2,A87peYAyqsf,10676,1862,0,1862
3,ABcUyezNjiu,3624,859,45,828
4,ACa6vpZuNzf,3341,616,3,613


In [86]:
# AGGREGATE IRS DATA BY FACILITY
irs_aggregate_df = irs_data.groupby('orgUnit', as_index =False).sum()
irs_aggregate_df.head()

,orgUnit,IRS-Total Population Protected,IRS-Total Structures Found,IRS-Total Structures Not Sprayed,IRS-Total Structures Sprayed
0,A2WG7HbvZ5m,7329,1983,1,1982
1,A7YT3iNUs5G,1760,329,6,323
2,A87peYAyqsf,10676,1862,0,1862
3,ABcUyezNjiu,3624,859,45,828
4,ACa6vpZuNzf,3341,616,3,613


In [87]:
# VIEW ORG UNIT DATA
org_unit_df.head()

,index,country,province,district,facility,facility_uid
0,0,Zambia,Central,Chibombo,ce Chamakubi Health Post,pXhz0PLiYZX
1,1,Zambia,Central,Chibombo,ce Chibombo Rural Health Centre,di3U5u7F8Y3
2,2,Zambia,Central,Chibombo,ce Chikobo Rural Health Centre,B5LpoYehUfI
3,3,Zambia,Central,Chibombo,ce Chikumbi ZNS Rural Health Centre,zAuN8bxMOMB
4,4,Zambia,Central,Chibombo,ce Chilochabalenge Health Post,g42i3akwlpj


In [88]:
# CLEAN MICROPLAN DATA
microplan_df.drop(["IRS- Campaign End Date","IRS- Campaign Start Date"], axis = 1, inplace=True)
microplan_df.reset_index(drop=True)
microplan_df.head()

,index,orgUnit,IRS-Total Eligible Structures,IRS-Total Targeted Eligible Structures,IRS-Total Targeted Population,IRS-Total Population
0,3,A2WG7HbvZ5m,1893,1893.0,7519.0,7519.0
1,333,A7YT3iNUs5G,170,170.0,1400.0,1899.0
2,218,A87peYAyqsf,2363,2363.0,7596.0,7596.0
3,50,A9BC0fz8cxX,3000,3000.0,9080.0,9080.0
4,119,ABcUyezNjiu,2133,1782.0,7604.0,7604.0


In [89]:
# MERGE MICROPLAN and IRS DATA
merged_df = pd.merge(irs_aggregate_df, microplan_df, on="orgUnit", how="left")
merged_df.rename(columns = {'orgUnit':'facility_uid'}, inplace = True)
merged_df.drop(columns=['index'], inplace=True)
merged_df.head()

,facility_uid,IRS-Total Population Protected,IRS-Total Structures Found,IRS-Total Structures Not Sprayed,IRS-Total Structures Sprayed,IRS-Total Eligible Structures,IRS-Total Targeted Eligible Structures,IRS-Total Targeted Population,IRS-Total Population
0,A2WG7HbvZ5m,7329,1983,1,1982,1893.0,1893.0,7519.0,7519.0
1,A7YT3iNUs5G,1760,329,6,323,170.0,170.0,1400.0,1899.0
2,A87peYAyqsf,10676,1862,0,1862,2363.0,2363.0,7596.0,7596.0
3,ABcUyezNjiu,3624,859,45,828,2133.0,1782.0,7604.0,7604.0
4,ACa6vpZuNzf,3341,616,3,613,1100.0,1100.0,3332.0,3332.0


In [90]:
# Merge Final DF with Org Unit DF
final_df = pd.merge(org_unit_df, merged_df, on="facility_uid", how="left").reset_index(drop=True)
final_df.dropna(subset=['IRS-Total Population Protected','IRS-Total Structures Found','IRS-Total Structures Not Sprayed', 'IRS-Total Structures Sprayed'],inplace=True)
# final_df.drop(["index_x"], axis = 1, inplace=True)
# final_df.drop(["index_y"], axis = 1, inplace=True)
final_df.drop(columns=['index'], inplace=True)

# Save the DataFrame to a SQL lite db
load_data("stage3_merged_data", final_df)

Data loaded to stage3_merged_data in DB


In [91]:
stage3_df = read_data("stage3_merged_data")
stage3_df.head(4)

Reading data from stage3_merged_data in DB


,index,country,province,district,facility,facility_uid,IRS-Total Population Protected,IRS-Total Structures Found,IRS-Total Structures Not Sprayed,IRS-Total Structures Sprayed,IRS-Total Eligible Structures,IRS-Total Targeted Eligible Structures,IRS-Total Targeted Population,IRS-Total Population
0,336,Zambia,Copperbelt,Chililabombwe,co Chimfunshi Rural Health Centre,J4Pfl0Qdjqs,4260.0,1037.0,181.0,865.0,NaN,NaN,NaN,NaN
1,337,Zambia,Copperbelt,Chililabombwe,co Fitobaula Health Post,n47ZlAEODR4,1743.0,433.0,80.0,354.0,672.0,642.0,3387.0,4415.0
2,338,Zambia,Copperbelt,Chililabombwe,co Fitobaula Health Post,Z9fLTlwKXmY,901.0,213.0,39.0,174.0,NaN,NaN,NaN,NaN
3,340,Zambia,Copperbelt,Chililabombwe,co Kakoso Urban Health Centre,lvn0izPxxfM,21933.0,4444.0,822.0,3623.0,5037.0,4175.0,16005.0,16186.0


#### PROVINCIAL PICTURE TABLE

In [92]:
stage3_df = read_data("stage3_merged_data")

# PROVINCIAL PICTURE:
provincial = stage3_df.copy()
provincial.drop(["country", "district","facility","facility_uid"], axis = 1, inplace=True)
provincial=provincial.groupby('province').sum()
# provincial.drop(["index_x"], axis = 1, inplace=True)

# Calculate % population protected.
provincial["pop_protected_perc"] = round(( provincial["IRS-Total Population Protected"] / provincial["IRS-Total Targeted Population"].fillna(0)) * 100)
provincial.loc[provincial['IRS-Total Targeted Population'] == 0, 'pop_protected_perc'] = 0

# Calculate Spray progress
provincial["spray_progress"] = round(( provincial["IRS-Total Structures Sprayed"] / provincial["IRS-Total Targeted Eligible Structures"].fillna(0)) * 100)
# Ensure that spray coverage is 0 if the denominator is 0
provincial.loc[provincial['IRS-Total Targeted Eligible Structures'] == 0, 'spray_progress'] = 0
provincial["spray_coverage"] = round(( provincial["IRS-Total Structures Sprayed"] / provincial["IRS-Total Structures Found"]) * 100)


# Save the DataFrame to a SQL lite db
load_data("provincial_progress", provincial)

Reading data from stage3_merged_data in DB
Data loaded to provincial_progress in DB


In [93]:
provincial = read_data("provincial_progress")
provincial.head(4)

Reading data from provincial_progress in DB


,province,index,IRS-Total Population Protected,IRS-Total Structures Found,IRS-Total Structures Not Sprayed,IRS-Total Structures Sprayed,IRS-Total Eligible Structures,IRS-Total Targeted Eligible Structures,IRS-Total Targeted Population,IRS-Total Population,pop_protected_perc,spray_progress,spray_coverage
0,Copperbelt,30915,474868.0,111102.0,13282.0,97926.0,109781.0,100844.00,601321.0,642295.0,79.0,97.0,88.0
1,Eastern,176146,1369888.0,353580.0,3319.0,350261.0,278646.0,270120.00,1130761.0,1138763.0,121.0,130.0,99.0
2,Luapula,193968,1283560.0,205120.0,6069.0,199174.0,186436.0,174638.00,853103.0,886951.0,150.0,114.0,97.0
3,Lusaka,25438,42062.0,12293.0,963.0,11379.0,32273.0,30515.22,150883.0,159072.0,28.0,37.0,93.0


#### DISTRICT PICTURE

In [99]:
stage3_df = read_data("stage3_merged_data")

# DISTRICT PICTURE:
district = stage3_df.copy()
district.drop(["country", "facility","facility_uid"], axis = 1, inplace=True)

district = district.groupby(['province', 'district'], as_index=False).sum()

# Calculate % population protected.
district["pop_protected_perc"] = round(( district["IRS-Total Population Protected"] / district["IRS-Total Targeted Population"].fillna(0)) * 100)
district.loc[district['IRS-Total Targeted Population'] == 0, 'pop_protected_perc'] = 0

# provincial.drop(["index_x"], axis = 1, inplace=True)
district["spray_progress"] = round(( district["IRS-Total Structures Sprayed"] / district["IRS-Total Targeted Eligible Structures"].fillna(0)) * 100)
# Ensure that spray coverage is 0 if the denominator is 0
district.loc[district['IRS-Total Targeted Eligible Structures'] == 0, 'spray_progress'] = 0

district["spray_coverage"] = round(( district["IRS-Total Structures Sprayed"] / district["IRS-Total Structures Found"]) * 100)


# Save the DataFrame to a SQL lite db
load_data("district_progress", district)

Reading data from stage3_merged_data in DB
Data loaded to district_progress in DB


In [100]:
district = read_data("district_progress")
district.head(4)

Reading data from district_progress in DB


,level_0,province,district,index,IRS-Total Population Protected,IRS-Total Structures Found,IRS-Total Structures Not Sprayed,IRS-Total Structures Sprayed,IRS-Total Eligible Structures,IRS-Total Targeted Eligible Structures,IRS-Total Targeted Population,IRS-Total Population,pop_protected_perc,spray_progress,spray_coverage
0,0,Copperbelt,Chililabombwe,3423,86630.0,19721.0,3846.0,15902.0,15871.0,14637.0,80771.0,82688.0,107.0,109.0,81.0
1,1,Copperbelt,Chingola,6628,188012.0,44012.0,8460.0,35575.0,47627.0,39924.0,243669.0,282726.0,77.0,89.0,81.0
2,2,Copperbelt,Kalulushi,1635,19014.0,4272.0,136.0,4136.0,0.0,0.0,0.0,0.0,0.0,0.0,97.0
3,3,Copperbelt,Lufwanyama,9226,89614.0,22421.0,360.0,22108.0,24407.0,24407.0,125882.0,125882.0,71.0,91.0,99.0


## REPORT GENERATION

In [19]:
import datetime
from jinja2 import Environment, FileSystemLoader, Template
import pandas as pd
import sqlite3
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import ssl
import pdfkit
from random import getrandbits, randint
import os

date = datetime.date.today()
current_year = datetime.datetime.now().year

con = sqlite3.connect("db/irs_final.db")

def make_report(con):
    try:
        # Load the data into a DataFrame
        provincial_df = read_data("provincial_progress")
        district_df_csv = read_data("district_progress")

        # Convert the DataFrame to a dictionary
        provincial_data = {}
        for _, row in district_df_csv.iterrows():
            province = row['province']
            if province not in provincial_data:
                provincial_data[province] = []
            provincial_data[province].append(row.to_dict())

        # Ensure the dataframe is not empty and convert it to a dictionary
        if not provincial_df.empty:
            national_data = provincial_df.to_dict(orient='index')
        else:
            raise ValueError("Provincial DataFrame is empty!")


        def format_with_commas(value):
            try:
                return "{:,}".format(value)
            except ValueError:
                return value

        data = {
                "subject": f"{current_year} IRS PROGRESS UPDATE",
                'data': national_data,
                'provincial_data':provincial_data,
            }



        # Load the Jinja2 template
        env = Environment(loader=FileSystemLoader('templates'))
        env.filters['format_with_commas'] = format_with_commas

        template = env.get_template('email_temp2.html')

        # Render the template
        html_content = template.render(data)


        # # TEST WRITING TO PDF.

        # write the html to file
        print("Generating HTML report....")
        with open(f"reports/report_{date}.html", 'wb') as file_:
            file_.write(html_content.encode("utf-8"))
        
        report_html = f"reports/report_{date}.html"

        #Define path to wkhtmltopdf.exe
        wkhtmltopdf_path = r'C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe'
        wkhtmltopdf_config = pdfkit.configuration(wkhtmltopdf=wkhtmltopdf_path)
        print("Converting HTML report to PDF")
        pdfkit.from_file(report_html, output_path=f"reports/report_{date}.pdf", configuration = wkhtmltopdf_config)
        
    except Exception as e:
        print("Something went wrong...!!!")

make_report(con)
print("Report generation complete")


Reading data from provincial_progress in DB
Reading data from district_progress in DB
Generating HTML report....
Converting HTML report to PDF
Report generation complete


In [17]:
from dotenv import load_dotenv, dotenv_values
import datetime
from jinja2 import Environment, FileSystemLoader, Template
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import ssl
import os

load_dotenv()

# SMTP_SERVER SETTINGS
smtp_port = 587
smtp_server = os.getenv("SMTP_SERVER")
gmail = os.getenv("SMTP_SERVER_EMAIL")
google_app_pass = os.getenv("SMTP_SERVER_PASS")

simple_email_context = ssl.create_default_context()

mail_to = ["",""]

date = datetime.date.today()
current_year = datetime.datetime.now().year

subject = f'{current_year} IRS Campaign Progress Update.'

def send_email(mail_to):


    for person in mail_to:
        body = """
        Hello there,

        Find attached the IRS report for the year 2024.
        """

        msg = MIMEMultipart()
        msg['From'] = gmail
        msg['To'] = person
        msg['Subject'] = subject

        msg.attach(MIMEText(body, 'plain'))

        #define file to attach
        filename = f'reports/report_{date}.pdf'

        # open the file in python as binary
        attachment = open(filename, 'rb') # r for read and b for binary

        # Encode it as base 64
        attachment_package = MIMEBase('application', 'octet-stream')
        attachment_package.set_payload(attachment.read())
        encoders.encode_base64(attachment_package)
        attachment_package.add_header('Content-Disposition', "attachment; filename= "+ filename)
        msg.attach(attachment_package)

        # Cast as string
        text = msg.as_string()


        # Connect with  server
        print("Connecting to server....")
        TIE_server = smtplib.SMTP(smtp_server, smtp_port)
        TIE_server.starttls()
        TIE_server.login(gmail, google_app_pass)
        print("Successfully connected to server.")
        print()

        # Send emails to "person" as list iterated
        print(f'Sending email to:{person}....')
        TIE_server.sendmail(gmail, person, text)
        print(f'Email sent to: {person}')
        print()

    TIE_server.quit()


send_email(mail_to)

        



Connecting to server....


KeyboardInterrupt: 

In [ ]:
stage3_df = pd.read_sql_query("SELECT * from stage1_irs_data", con)